# Project: Wrangling and Analyze Data

## Import libraries

In [15]:
import pandas as pd
import requests
import tweepy
import time
import json

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [4]:
twitter_archive = pd.read_csv("twitter-archive-enhanced.csv")
twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [7]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
res = requests.get(url, allow_redirects=True)
with open('image_prediction.tsv', 'wb') as file:
    file.write(res.content)
print(res)

<Response [200]>


3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [111]:
consumer_key = 'DnVOLqKhguKff9Z6hO8t2cktH'
consumer_secret = 'RcNFzk8WL4HJcPGz1NpVOar4j7g02ox4iWUKclnl4ekGJeh5HR'
access_token = '422293955-LpuVyFhnXOcAHkw2a4Tn0LNzn4SdFM8Q4J0BAia8'
access_secret = '9b0tKWQRKma7rzw55lHsOajtBMTP4fN5jrbMop5qnQxGR'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

fails_dict = {}
start = time.time()
count = 0

downloaded_tweets = []

with open('tweet_json.txt', 'r') as tweets_status:
    for tweet in tweets_status.readlines():
        tweet_json = json.loads(tweet)
        downloaded_tweets.append(tweet_json["id"])

with open('tweet_json.txt', 'a') as tweets_file:
    for tweet_id in twitter_archive.tweet_id:
        count = count + 1
        if tweet_id not in downloaded_tweets:
            print("Trying tweet {}".format(count))
            try:
                tweet = api.get_status(tweet_id, tweet_mode='extended')
                tweet_json = json.dumps(tweet._json)    
                tweets_file.write(tweet_json)
                tweets_file.write('\n')
            except tweepy.TweepyException as e:
                print("{} Failed: {}".format(tweet_id, e))
                fails_dict[tweet_id] = str(e)
                pass
        else:
            pass
#             print("Tweet {} already downloaded".format(tweet_id))


end = time.time()

print(end - start)

with open('tweet_json_failed.txt', 'w') as failed_tweets_file:
    print(len(fails_dict))
    
    failed_ids = json.dumps(fails_dict)    
    failed_tweets_file.write(failed_ids)


                


Trying tweet 20
888202515573088257 Failed: 404 Not Found
144 - No status found with that ID.
Trying tweet 96
873697596434513921 Failed: 404 Not Found
144 - No status found with that ID.
Trying tweet 102
872668790621863937 Failed: 404 Not Found
144 - No status found with that ID.
Trying tweet 105
872261713294495745 Failed: 404 Not Found
144 - No status found with that ID.
Trying tweet 119
869988702071779329 Failed: 404 Not Found
144 - No status found with that ID.
Trying tweet 133
866816280283807744 Failed: 404 Not Found
144 - No status found with that ID.
Trying tweet 156
861769973181624320 Failed: 404 Not Found
144 - No status found with that ID.
Trying tweet 183
856602993587888130 Failed: 404 Not Found
144 - No status found with that ID.
Trying tweet 186
856330835276025856 Failed: 404 Not Found
34 - Sorry, that page does not exist.
Trying tweet 212
851953902622658560 Failed: 404 Not Found
144 - No status found with that ID.
Trying tweet 213
851861385021730816 Failed: 404 Not Found
14

## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



### Quality issues
1.

2.

3.

4.

5.

6.

7.

8.

### Tidiness issues
1.

2.

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [ ]:
# Make copies of original pieces of data


### Issue #1:

#### Define:

#### Code

#### Test

### Issue #2:

#### Define

#### Code

#### Test

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.

2.

3.

### Visualization